**

### **Step 1: Choose and Load a Dataset**
- Pick a dataset (e.g., CIFAR-10, Flowers, Fruits, or Animals).  
- You can download a dataset from **Kaggle** or use built-in ones from **TensorFlow/Keras** (`tf.keras.datasets`).  
- Load the dataset and split it into **training** and **testing** sets.  

### **Step 2: Load Pre-trained VGG16 Model**
- Use `keras.applications.VGG16` with **pre-trained ImageNet weights**.  
- **Remove the top layer** (fully connected layer for classification).  
- **Add a new fully connected layer** that matches your dataset’s number of classes.  

### **Step 3: Preprocess Images**
- Resize all images to **224×224 pixels** (VGG16 input size).  
- Normalize pixel values **between 0 and 1** (divide by 255).  
- Use `ImageDataGenerator` for easy preprocessing.  

### **Step 4: Freeze Convolutional Layers & Train Custom Layers**
- **Freeze all convolutional layers** (so they don’t update weights).  
- Train **only the newly added fully connected layers**.  
- Use **Adam or SGD optimizer**.  
- Train for **at least 5–10 epochs** and observe accuracy.  

### **Step 5: Evaluate Model Performance**
- Use **test set** to check accuracy and loss.  
- Generate **a confusion matrix** to analyze predictions.  
- Generate **a classification report** (precision, recall, F1-score).  

### **Step 6: Apply Data Augmentation**
- Use `ImageDataGenerator` for:  
  - **Rotation**  
  - **Flipping**  
  - **Zooming**  
- Train the model again and compare results with/without augmentation.  

### **Step 7: Compare VGG16 with Another Model**
- Choose **another pre-trained model** (e.g., MobileNet or ResNet).  
- Follow the **same process** (load, modify, train, and evaluate).  
- Compare **accuracy, speed, and performance**.  

### **Step 8: Submit the Notebook**
- **Summarize findings** in a Jupyter Notebook (`.ipynb`).  
- Show **comparison grow if you need code snippets or any clarifications! 🚀🔥

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Step 1: Load CIFAR-10 Dataset
dataset = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = dataset.load_data()

# Use a smaller subset
subset_size = 1000  # Choose a smaller number
x_train, y_train = x_train[:subset_size], y_train[:subset_size]
x_test, y_test = x_test[:subset_size // 10], y_test[:subset_size // 10]

In [5]:
print(f"Training set shape: {x_train.shape}")
print(f"Test set shape: {x_test.shape}")

Training set shape: (1000, 32, 32, 3)
Test set shape: (100, 32, 32, 3)


In [6]:
# Resize images to 224x224 (VGG16 input size)
x_train = np.array([tf.image.resize(img, (224, 224)).numpy() for img in x_train])
x_test = np.array([tf.image.resize(img, (224, 224)).numpy() for img in x_test])

In [7]:
x_train.shape

(1000, 224, 224, 3)

In [8]:
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

In [9]:
x_train.shape

(1000, 224, 224, 3)

In [10]:
# Step 2: Load Pretrained VGG16 (without top layer)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_base.trainable = False  # Freeze convolutional layers

In [11]:
# Step 3: Add Custom Fully Connected Layers
x = Flatten()(vgg16_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
out_layer = Dense(10, activation='softmax')(x)  # 10 classes in CIFAR-10

model = Model(inputs=vgg16_base.input, outputs=out_layer)

In [12]:
# Step 4: Compile Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Train the Model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=3, batch_size=32)

Epoch 1/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 416s 13s/step - accuracy: 0.1100 - loss: 3.3420 - val_accuracy: 0.2500 - val_loss: 2.1433
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 1420s 45s/step - accuracy: 0.1798 - loss: 2.1606 - val_accuracy: 0.3100 - val_loss: 1.9544
Epoch 3/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 501s 16s/step - accuracy: 0.2437 - loss: 2.0570 - val_accuracy: 0.3300 - val_loss: 1.9391


In [ ]:
# Step 6: Evaluate Model Performance
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
# Generate Classification Report & Confusion Matrix
y_pred = np.argmax(model.predict(x_test), axis=1)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:

# Step 7: Apply Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_aug = data_gen.flow(x_train, y_train, batch_size=32)

In [ ]:

# Train Again with Augmentation
model.fit(train_aug, validation_data=(x_test, y_test), epochs=10, batch_size=32)

# Step 8: Compare with MobileNetV2
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenet_base.trainable = False

x = Flatten()(mobilenet_base.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
out_layer = Dense(10, activation='softmax')(x)


In [ ]:
mobilenet_model = Model(inputs=mobilenet_base.input, outputs=out_layer)
mobilenet_model.compile(optimizer=Adam(learning_rate=0.001),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

# Train MobileNet Model
mobilenet_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)




In [ ]:
# Compare Performance
loss_vgg16, acc_vgg16 = model.evaluate(x_test, y_test)
loss_mobilenet, acc_mobilenet = mobilenet_model.evaluate(x_test, y_test)

print(f"VGG16 Test Accuracy: {acc_vgg16:.4f}")
print(f"MobileNetV2 Test Accuracy: {acc_mobilenet:.4f}")